In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
import glob
import re
import os
from io import StringIO
from pdfminer.pdfpage import PDFPage
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.layout import LAParams
from pdfminer.converter import TextConverter
import PyPDF2

In [18]:
df = pd.read_csv('/Users/alisdghnia/Desktop/Files Texts Dataframe.csv', index_col=False)

In [19]:
df = df.rename(columns={'Unnamed: 0':'File Name'})

In [4]:
df = df.set_index('File Name')

In [20]:
df

,File Name,Text Corpus
0,WeTrust1,wetrust whitepaper table of contents \n\n \n\n...
1,Libra4,state machine replication in the libra\n\nbloc...
2,B2BX1,23\n\n
3,Bytom1,bytomv1.0 ...
4,Cindicator1,hybrid intelligence for\n\neective asset mana...
...,...,...
551,CPChain1,\t\n\ncyber-physical chain (cpchain) whitepape...
552,Presearch2,whitepaper\n\nthe community-powered search eng...
553,Electra1,"white paper\n\nrelease 1.0 / january 31, 2018\..."
554,Blockmason1,the credit protocol whitepaper v1.0.1\n\nblock...


In [6]:
print(df.loc['WeTrust1','Text Corpus'])

wetrust whitepaper table of contents 

 

abstract 

vision 

market review 

trusted lending circle, a simple reciprocal aid organization: 

wetrust product ecosystem 

wetrust people ecosystem (added after daniel z review) 

app token 

app token usage within trusted lending circles 
scaling up trusted lending circles 

wetrust challenges and solutions 

product road map/ budget 

budget structure 

go to market strategy (added after daniel z review) 

blockchain benefits 

potential market size 

technical aspects 

the first dapp - trusted lending circle (aka rosca): 

why start with trusted lending circle as wetrust’s first dapp? 
deposits and withdrawals 
trusted lending circle-rosca dapp walkthrough 

team 

glossary 

appendix 

references 
 
 
 
 
 

george li, patrick long, ron merom, tom nash, mivsam yekutiel 

 

www.wetrust.io 

 

 

abstract 
 
wetrust is a collaborative savings, lending and insurance platform that is autonomous, 
agnostic, frictionless, and decentraliz

In [21]:
def clean_text(text):
    # remove numbers
    text_nonum = re.sub(r'\d+', '', str(text))
    # remove punctuations and convert characters to lower case
    text_nopunct = "".join([char.lower() for char in text_nonum if char not in string.punctuation]) 
    # substitute multiple whitespace with single whitespace
    # Also, removes leading and trailing whitespaces
    text_no_doublespace = re.sub('\s+', ' ', text_nopunct).strip()
    return text_no_doublespace


def remove_stops(text, stops):
    text = re.sub(r"AC\/\d{1,4}\/\d{1,4}", "", str(text))
    words = text.split()
    final = []
    for word in words:
        if word not in stops:
            final.append(word)
    final = " ".join(final)
    final = final.translate(str.maketrans("", "", string.punctuation))
    final = "".join([i for i in final if not i.isdigit()])
    while "  " in final:
        final = final.replace("  ", " ")
    return (final)

In [22]:
small_df = df.iloc[0:10]

In [23]:
small_df

,File Name,Text Corpus
0,WeTrust1,wetrust whitepaper table of contents \n\n \n\n...
1,Libra4,state machine replication in the libra\n\nbloc...
2,B2BX1,23\n\n
3,Bytom1,bytomv1.0 ...
4,Cindicator1,hybrid intelligence for\n\neective asset mana...
5,iXledger1,\n\nan​ ​insurance​ ​marketplace​ ​based​ ​on...
6,FARAD1,farad:\n\ncommoditising forward purchase contr...
7,DomRaider1,2\n\nadvance warning\n\nthe initial coin offe...
8,Zeusshield1,zeusshield system\n\nzeusshield smart insuranc...
9,Steem1,"\n \n\n \n\nsteem \n\nan​ ​incentivized,​ ​b..."


In [10]:
texts = small_df.iloc[:,0].tolist()

In [ ]:
texts = remove_stops(texts, 'english')
texts = clean_text(texts)
texts

In [24]:
texts = df.iloc[:,0].tolist()

In [25]:
texts = remove_stops(texts, 'english')
texts = clean_text(texts)

In [26]:
texts = [texts]

In [27]:
vectorizer = TfidfVectorizer()
tfidf_model = vectorizer.fit_transform(texts)

In [28]:
tfidf = pd.DataFrame(tfidf_model.toarray(), columns = vectorizer.get_feature_names())
tfidf

/Users/alisdghnia/.pyenv/versions/miniforge3-4.10.3-10/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,aave,abbc,achain,adex,aditus,adtoken,aelf,aeron,aeternity,agrello,...,zap,zcash,zclassic,zcoin,zeepin,zencash,zerocash,zerocoin,zeusshield,zilliqa
0,0.064617,0.032309,0.032309,0.032309,0.032309,0.032309,0.032309,0.032309,0.064617,0.032309,...,0.032309,0.064617,0.032309,0.032309,0.032309,0.032309,0.064617,0.032309,0.032309,0.064617


In [29]:
corpustext = df['Text Corpus']
print(corpustext)

0      wetrust whitepaper table of contents \n\n \n\n...
1      state machine replication in the libra\n\nbloc...
2       23\n\n
3      bytomv1.0                                     ...
4      hybrid intelligence for\n\neective asset mana...
                             ...                        
551    \t\n\ncyber-physical chain (cpchain) whitepape...
552    whitepaper\n\nthe community-powered search eng...
553    white paper\n\nrelease 1.0 / january 31, 2018\...
554    the credit protocol whitepaper v1.0.1\n\nblock...
555    “the full potential of connected devices is on...
Name: Text Corpus, Length: 556, dtype: object


In [30]:
corpustext = [text for text in corpustext]

In [31]:
corpustext = remove_stops(corpustext, 'english')
corpustext = clean_text(corpustext)

In [33]:
corpustext = [corpustext]

In [34]:
vectorizer = TfidfVectorizer()
tfidf_model = vectorizer.fit_transform(corpustext)
tfidf = pd.DataFrame(tfidf_model.toarray(), columns = vectorizer.get_feature_names())
tfidf

/Users/alisdghnia/.pyenv/versions/miniforge3-4.10.3-10/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,aa,aaa,aaabnnshared,aaacid,aaai,aaave,aab,aaba,aabcde,aac,...,𝜋jgá,𝜎𝐸σ,𝜒nnfinally,𝜔am,𝜔am𝑉am𝜔xn𝑉xn𝜔en𝑉en,𝜔en,𝜔xn,𝜔𝑉,𝞺i,𝞺in
0,0.000287,0.00003,0.000003,0.000005,0.000003,0.000003,0.000003,0.000014,0.000003,0.000003,...,0.000003,0.000003,0.000003,0.000005,0.000003,0.000005,0.000005,0.000008,0.000016,0.000005
